In [17]:
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader("final_output02.csv")
docs = loader.load()
docs

[Document(page_content='Link: https://www.udemy.com/course/complete-python-bootcamp/\nCourse Name: The Complete Python Bootcamp From Zero to Hero in Python\ntotal hours: 22.0\nrating: 4.6\nlevel: All Levels', metadata={'source': 'final_output02.csv', 'row': 0}),
 Document(page_content='Link: https://www.udemy.com/course/the-complete-web-development-bootcamp/\nCourse Name: The Complete 2024 Web Development Bootcamp\ntotal hours: 61.5\nrating: 4.7\nlevel: All Levels', metadata={'source': 'final_output02.csv', 'row': 1}),
 Document(page_content='Link: https://www.udemy.com/course/100-days-of-code/\nCourse Name: 100 Days of Code: The Complete Python Pro Bootcamp\ntotal hours: 58.5\nrating: 4.7\nlevel: All Levels', metadata={'source': 'final_output02.csv', 'row': 2}),
 Document(page_content='Link: https://www.udemy.com/udemy-business/request-demo-in-mx/?ref=right-rail&locale=en_US\nCourse Name: Try Udemy Business\ntotal hours: \nrating: \nlevel: ', metadata={'source': 'final_output02.csv', 

In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 10)
documents = text_splitter.split_documents(docs)

In [19]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

In [20]:
db = FAISS.from_documents(documents, OllamaEmbeddings())

In [21]:
db

In [22]:
from langchain_community.llms import Ollama

llm = Ollama(model= "llama2")
llm

Ollama()

In [38]:
#chat prompt

from langchain_core.prompts import ChatPromptTemplate



prompt_json= ChatPromptTemplate.from_template(
    """  Act as a helpful Talent Transformation assistant and suggest the best courses to take based on the 
    provided context and skills. Additionally, provide the course link and total hours in JSON format. 
    Take your time to think step by step before providing any answer. 
    If there is a course suggestion, extract it as JSON with the following keys:
        - "Course Name": Name of the course.
        - "Course Link" : Link of the course.
        - "Total Hours": Total hours of the course.
    
    Strickly follow the context.
    <context>
    {context}
    <context>

    skills : {input}
"""
)

In [25]:
#retriever

retriever = db.as_retriever()

In [26]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

setup = RunnableParallel(context=retriever, input = RunnablePassthrough())

In [27]:
from langchain_core.output_parsers import JsonOutputParser

parser_js = JsonOutputParser()

In [40]:
chain_js = setup | prompt_json | llm 

In [41]:
chain_js.invoke("my skills are python and javascript")

'Based on the provided context and your skills in Python and JavaScript, I would suggest the following courses:\n\n1. Vue - The Complete Guide (incl. Router & Composition): This course is a comprehensive guide to Vue.js, covering both the fundamentals and advanced topics such as routing and composition. As a beginner-level developer, this course will provide you with a solid understanding of Vue.js and help you build complex applications. Total hours: 32.0 | Rating: 4.8 | Level: All Levels\n\nCourse Link: <https://www.udemy.com/course/vuejs-2-the-complete-guide/>\n\n2. Python for Beginners: Learn Python Programming: This course is designed to teach beginners the basics of Python programming. It covers topics such as data types, functions, and file input/output. As a Python developer, this course will help you brush up on your fundamentals and improve your problem-solving skills. Total hours: 3.0 | Rating: 4.5 | Level: Beginner\n\nCourse Link: <https://www.udemy.com/course/python-progra

In [32]:
chain_js_01 = setup | prompt_json | llm 

In [31]:
chain_js_01.invoke("my skill is python")

'Based on your input, I suggest the following courses to take based on your skills and context:\n\n1. Python for Beginners: Learn Python Programming - Udemy Course (Total hours: 3, Rating: 4.5, Level: Beginner)\nThis course is a great starting point for beginners who want to learn Python programming. The course covers the basics of Python programming and provides practical exercises to help you solidify your knowledge. As someone with skills in Python, this course will help you improve your skills and expand your knowledge in the language.\n\nCourse Link: <https://www.udemy.com/course/python-programming-projects/>\n\n2. The Complete Java Course - Udemy Course (Total hours: 68.5, Rating: 4.7, Level: Expert)\nWhile you may not have expert-level skills in Java yet, this course covers everything from the basics to advanced topics, providing a comprehensive learning experience. As someone with skills in Python, you\'ll find many similarities between the two languages and can apply your know

In [36]:
chain_js_02 = setup | prompt_json | llm | parser_js 

In [37]:
chain_js_02.invoke("my skill is python")

OutputParserException: Invalid json output: Based on the given context and your skill in Python, I would recommend the following courses:

1. "Python for Beginners: Learn Python Programming" - This course is a great starting point for beginners, covering the basics of Python programming. It has a total of 3 hours of content and a rating of 4.5/5. Since you're looking to improve your Python skills, this course would be a good fit for you.

Course Link: <https://www.udemy.com/course/python-programming-projects/>
Total Hours: 3.0 hours
Rating: 4.5/5
Level: Beginner

2. "The Complete Java Course" - This course is designed for beginners and covers the basics of Java programming. It has a total of 68.5 hours of content and a rating of 4.7/5. Since you're looking to improve your skills in Python, this course could still be beneficial as it covers fundamental programming concepts that are applicable across different languages.

Course Link: <https://www.udemy.com/course/the-complete-javascript